
# Relocation Recommendation from Delhi to Toronto Using Neighborhood Clustering 

<img src="https://www.immigration.ca/wp-content/uploads/2018/08/india-canada_98333203.jpeg" width="500" height="300" />

----

## Table of contents

* [Introduction: Problem Equation](#introduction)
* [Data Acquisition and prepration](#data)
* [Foursquare API Introduction](#Foursquare)
* [Exploratory Data Analysis : Normalization of the data for clustering](#Explore)
* [Obtaining Similar Table for the Near-to-Work/School Boroughs in Toronto ](#Similar)
* [Analyzing the Venues for Clustering](#Analysis_cluster)
* [Applying the clustering algorithm](#K-means)
* [Visualizing Clusters](#VisualizingClusters)
* [Analyzing Clusters](#Analyzing_clusters)
* [RECOMENDATIONS](#recommendation)
* [Results and Discussion](#Results)



---

# Introduction: Problem Equation

#####  In this project, we will try to find an optimal location for someone relocating from the city of Delhi(India) to Toronto(Canada) 

Specifically, this report will be a suggestion to those students and employees to whom relocating does bring a huge change in lifestyle and they want to adapt to their new-habitat as easily as possible. Finding a suitable location for relocation for anyone in major cities like Toronto proves to be a daunting task. Various factors such as Public Facilities, Favorite Cuisine, restaurants, religious practices, etc matters to the people. Hence, migrant can bolster their decisions using the descriptive and predictive capabilities of data science.

We need to find locations(Neighborhood) that have a similar nearby venues in Toronto compared to persons native Neighborhood in Delhi. Also, we need locations that is close to Work/Study location in Toronto. We will use our data science powers to generate a few most promising neighborhoods based on this criteria. The result will be published within the notebook in Tabular and map from

<h3>Data Acquisition and prepration</h3><a name='data'></a>

##### Based on definition of our problem, factors that will influence our decision are:

   * Office/University Borough location in the city of Toronto.
   * Similar neighborhood in Toronto than in Delhi.
   * Distance of neighborhood from popular neighborhoods.

##### In our project we will:

   * acquire the names and boroughs of the neighborhoods by scrapping a wikipedia pages.
   * After we have got the names of all the neighborhoods, we will geocode them using the geopy.geocoder (Nominatim).
   * Next, we use the foursquare API to find all types of restaurants within a 500 meter radius for every neighborhood.
   * Apply K-Means Cluster Analysis for the Comparison between various neighbourhoods and grouping them in clusters



In [1]:
##  importing useful Python Libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # to send url requests

!pip install html5lib ## (For web scraping)

! pip install BeautifulSoup4
from bs4 import BeautifulSoup # library to scrape the Webpages

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# impoerting file format handling libraries
import re
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Clustring algorithim library
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from IPython.core.display import display, HTML
from IPython.display import display_html, Markdown

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library


print('Libraries imported.')

     |████████████████████████████████| 112kB 18.4MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1

---
#### We will Scrape the Data of Delhi from a Wikipedia page into the Notebook and convert it to usable Data Frame for Processing and Cleaning
---

<img src="http://mapsof.net/uploads/static-maps/Delhi_Location.PNG" width="350" height="500" />

#### Obtaining Delhi's Lattitude and Longitude
-------

In [2]:
address = 'New Delhi'
geolocator = Nominatim(user_agent="Delhi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New_delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New_delhi are 28.6141793, 77.2022662.


#### Scrapping the Wikipedia Page

----------------------------------




In [3]:
#accessing the web page by Http request made by requests library


req = requests.get("https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi").text
soup = BeautifulSoup(req, 'html5lib')
div = soup.find('div', class_="mw-parser-output" )
print("web Page Imported")


web Page Imported


In [4]:
   #Code to extract the relevent data from the request object using beautiful soup
    
    
data = pd.DataFrame(columns=['Borough','Neighborhood'])
i=-1
flag = False
no=0
prev_borough = None
for child in div.children:
    if child.name:
        span = child.find('span')
        if span!=-1 and span is not None:
             try:
                 if span['class'][0] == 'mw-headline' and child.a.text!='edit':
                     prev_borough = child.a.text
                     i+=1
                     flag = True
                     continue
             except KeyError:
                 continue
        if child.name=='ul' and flag==True:
             neighborhood = []
             for ch in child.children:
                
                 try:
                     data.loc[no]=[prev_borough,ch.text]
                     no+=1
                 except AttributeError:
                     pass
        flag = False
 data[50:60]

#### Geocoding every neighborhood

---

In [5]:
 lat_lng = pd.DataFrame(columns=['latitude','longitude'])
 geolocator = Nominatim(user_agent="Delhi_explorer")
 for i in range(184):
     address = data['Neighborhood'].loc[i]+',Delhi'
     try: 
         location = geolocator.geocode(address)
         lat_lng.loc[i]=[location.latitude,location.longitude]
     except AttributeError:
         continue
            

#### Merging the Latitude and Longitude Values with actual Neighborhood Values

---

In [6]:
df1 = data
df2 = lat_lng
delhi_neighborhood_data = pd.concat([df1, df2], axis=1)

### below code was for my own use to download my data to my local maschine(YOU CAN IGNORE)

#from IPython.display import HTML
#import base64 

#def create_download_link( df, title = "Download CSV file", filename = "delhi_neighborhood_data.csv"):  
  #  csv = df.to_csv()
   # b64 = base64.b64encode(csv.encode())
   # payload = b64.decode()
   # html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
   # html = html.format(payload=payload,title=title,filename=filename)
    #return HTML(html)

#create_download_link(delhi_neighborhood_data)

delhi_neighborhood_data.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.714401,77.167288
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
4,North West Delhi,Begum Pur,NaN,NaN


#### Processing the Scraped Data


In [7]:
## The following code is commented because sometimes Geocode doesnot work and I has saved 'delhi_neighbourhood_data' in working directory to dorectly pick 

#delhi_neighborhood_data = pd.read_csv('delhi_neighborhood_data.csv')

#delhi_neighborhood_data.reset_index(inplace=True)
#delhi_neighborhood_data.drop(['index','Unnamed: 0'], axis=1, inplace=True)
#delhi_neighborhood_data.drop('level_0', axis=1, inplace=True)

#delhi_neighborhood_data.drop(1, axis=0, inplace=True)
#delhi_neighborhood_data.reset_index(inplace=True)
#delhi_neighborhood_data.drop(['index'], axis=1, inplace=True)

delhi_neighborhood_data.dropna(inplace=True)
delhi_neighborhood_data.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.714401,77.167288
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
5,North West Delhi,Dhaka,28.708698,77.205749


#### Visualing the obtained data set
----

In [8]:
delhiData = delhi_neighborhood_data
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(delhiData['latitude'], delhiData['longitude'], delhiData['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)  
display(Markdown('### A folium map is being dispayed here, which mostly doesnot show on GITHUB '))  
map_delhi

### A folium map is being dispayed here, which mostly doesnot show on GITHUB 

Now that we have obtained the location of every neighborhood we can use the foursquare API to find the location of nearby Venues

----
<h3> Foursquare API Introduction  </h3> <a name='Foursquare'></a>

The Foursquare Places API provides location based experiences with diverse information about venues, users, photos, and check-ins. The API supports real time access to places, Snap-to-Place that assigns users to specific locations, and Geo-tag.

Here we are using the explore api call and filtering the search only to find venues that are identified as Venues.

In [9]:
CLIENT_ID = 'R0WQO1KK2OAXFTDGCJ3FYHCIFHP5ZCUQFPTUHWIVQDUPJ3EU' # your Foursquare ID
CLIENT_SECRET = 'X10444DCZGJFYYJOU3JLOEGHT5E4BNJAQ33JWMXU5NVB4HXG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100

print('Credentails Registered')

Your credentails:
CLIENT_ID: R0WQO1KK2OAXFTDGCJ3FYHCIFHP5ZCUQFPTUHWIVQDUPJ3EU
CLIENT_SECRET:X10444DCZGJFYYJOU3JLOEGHT5E4BNJAQ33JWMXU5NVB4HXG
Credentails Registered


----
#### Now we will Inspect Different Neighborhoods in the city of Delhi by gathering Venues nearby and within that neighborhood



In [10]:
##  a function to gather and clean venues nearby all the neighborhoods in DELHI

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
delhi_venues = getNearbyVenues(names=delhiData['Neighborhood'],
                                    latitudes=delhiData['latitude'],
                                    longitudes=delhiData['longitude']
                                   )
delhi_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.714401,77.167288,Love Back Spells,28.715262,77.166102,Astrologer
1,Adarsh Nagar,28.714401,77.167288,Vishyavidyalaya Metro Station@Entry gate #1 n ...,28.715596,77.170981,Train Station
2,Adarsh Nagar,28.714401,77.167288,Adarsh Nagar Metro Station,28.716598,77.170436,Light Rail Station
3,Adarsh Nagar,28.714401,77.167288,The Parkour Academy India,28.716500,77.162900,Gym / Fitness Center
4,Ashok Vihar,28.699453,77.184826,SNAP FITNESS 24×7 GYM,28.700180,77.184582,Athletics & Sports


----
#### Summary till now

* The first was the data set(delhi_data) that contained the borough, name, Latitude and Longitude of all the major Neighborhoods of Delhi
* And, the second data set(delhi_venues) contained the geographical information pertinent to all the major venues in these neighborhoods of delhi
<br><br>


---
<h3> Exploratory Data Analysis : Normalization of the data for clustering </h3> <a name='Explore'></a>

----

In [12]:
# one hot encoding
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")
delhi_onehot.drop('Neighborhood', axis=1, inplace=True)

# add neighborhood column back to dataframe
delhi_onehot=pd.concat([delhi_venues['Neighborhood'], delhi_onehot], axis=1) 

delhi_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Café,Campground,Candy Store,Castle,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,North Indian Restaurant,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Restaurant,Road,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Weight Loss Center,Wine Bar,Women's Store
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [13]:
delhi_onehot.shape

(1192, 173)

In [14]:
#To be used while Generating Graphs
#Grouping all with same neighbourhoods together

delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Café,Campground,Candy Store,Castle,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,North Indian Restaurant,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Restaurant,Road,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Weight Loss Center,Wine Bar,Women's Store
0,Adarsh Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.181818,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0

In [15]:
delhi_grouped.shape

(155, 173)

----
#### Obtaining Data frame for most common venues nearby neighborhoods of Delhi.
----

In [16]:
### Defining a function for finding most common venues in a neighborhood


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = delhi_grouped['Neighborhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Adarsh Nagar,Train Station,Gym / Fitness Center,Light Rail Station,Astrologer,Women's Store,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Fish & Chips Shop,Garden
1,Alaknanda,Indian Restaurant,BBQ Joint,Steakhouse,New American Restaurant,Pizza Place,Coffee Shop,Food & Drink Shop,Restaurant,Middle Eastern Restaurant,French Restaurant,Food Truck,Food Court,Food,Flea Market,Fast Food Restaurant
2,Anand Vihar,Fast Food Restaurant,Pizza Place,Movie Theater,Multiplex,Women's Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Department Store,Falafel Restaurant
3,Ashok Nagar,Fast Food Restaurant,Ice Cream Shop,Café,Clothing Store,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Women's Store,Furniture / Home Store,Farmers Market
4,Ashok Vihar,Diner,Athletics & Sports,Bakery,Smoke Shop,Health & Beauty Service,Hotel Bar,Electronics Store,Donut Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food


-------

-----

 <h3> Obtaining Similar Table for the Near-to-Work/School Boroughs in Toronto </h3> <a name='Similar'></a> 


We have the desired Delhi_Venues Data Frame now, our methodology involves making a similar Data Frame for the locations of Toronto which are nearby our migrant's place our school. We will for our project purpose assume that our Migrant prefers Boroughs in TORONTO by our migrant are the adjacent Borough of North York and Scarborough

-----


<img src="https://upload.wikimedia.org/wikipedia/commons/9/9a/Toronto_map.png" width="500" height="700" />

----

In [18]:
##Reading the Toronto CSV file from Working Directory 
## This data Contains the Cleaned Toronto_Data_Needed from the previous assignments on Toronto Cluters

Toronto_Data_Needed=pd.read_csv('TORONTO_DATA_Needed.csv')

In [19]:
Toronto_Data_Needed.head()

,Unnamed: 0,Postal code,Borough,Neighborhood,Latitude,Longitude
0,0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,1,M2J,North York,"Fairview , Henry Farm , Oriole",43.778517,-79.346556
2,2,M2K,North York,Bayview Village,43.786947,-79.385975
3,3,M2L,North York,"York Mills , Silver Hills",43.757490,-79.374714
4,4,M2M,North York,"Willowdale , Newtonbrook",43.789053,-79.408493


In [20]:
Toronto_Data_Needed.drop("Unnamed: 0", axis=1, inplace=True)
Toronto_Data_Needed.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview , Henry Farm , Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills , Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale , Newtonbrook",43.789053,-79.408493


----
#### Now that we have the data we will call the Foursqaure API functions to obtain the Venues in North York and Scarborough (TORONTO)

----

In [21]:
## Using the GetNearbyVenues funtions defined previously in teh notebook

Toronto_Venues = getNearbyVenues(names=Toronto_Data_Needed['Neighborhood'],
                                   latitudes=Toronto_Data_Needed['Latitude'],
                                   longitudes=Toronto_Data_Needed['Longitude']
                                  )
Toronto_Venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


In [22]:
print('There are {} uniques categories.'.format(len(Toronto_Venues['Venue Category'].unique()))) ## jsut to check the unique values

There are 121 uniques categories.


#### Exploratory Data Analysis : Normalization of the data for clustering
----

In [23]:
## ANALYSING ALL THE GROUPED NEIGHBORHOODS


# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Stadium,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,History Museum,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
## GROUPING by all the Neighbourhoodsand finding the avg visit to one Venue in that Neighborhood by mean function

Toronto_Grouped=Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_Grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Stadium,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,History Museum,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Agincourt,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.250000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.25,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.0,0.0,0.0,0.000000,0.05,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.100000,0.00,0.000000,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.05,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.05,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.050000,0.0,0.050000,0.05,0.00,0.0,0.0,0.0,0.0,0.05,0.0,0.050000,0.0,0.000000,0.0,0.0,0.0,0.0,0.05,0.0,0.0
2,Bayview Village,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,0.0,0.25,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.25,0.0,0.000000,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,"Bedford Park , Lawrence Manor East",0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.041667,0.00,0.0,0.0,0.0,0.0,0.0,0.041667,0.041667,0.0,0.00,0.0,0.0,0.083333,0.00,0.041667,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.041667,0.041667,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0


---

#### Data frame of frequently visited venues nearby our Boroughs of Toronto, as we had for Delhi !

----

In [25]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Neighborhoods_venues_sorted['Neighborhood'] = Toronto_Grouped['Neighborhood']

for ind in np.arange(Toronto_Grouped.shape[0]):
    Neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_Grouped.iloc[ind, :], num_top_venues)

Neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Agincourt,Skating Rink,Latin American Restaurant,Lounge,Breakfast Spot,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space
1,"Bathurst Manor , Wilson Heights , Downsview North",Bank,Coffee Shop,Supermarket,Bridal Shop,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Pharmacy,Deli / Bodega,Diner,Middle Eastern Restaurant,Ice Cream Shop,Sushi Restaurant,Fried Chicken Joint
2,Bayview Village,Bank,Café,Chinese Restaurant,Japanese Restaurant,Women's Store,Event Space,Electronics Store,Dog Run,Discount Store,Diner,Food & Drink Shop,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
3,"Bedford Park , Lawrence Manor East",Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Hobby Shop,Breakfast Spot,Grocery Store,Indian Restaurant,Juice Bar,Liquor Store,Comfort Food Restaurant,Pharmacy,Café,Pizza Place,Butcher
4,"Birch Cliff , Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space


----
----
<h3> Analyzing the Venues for Clustering </h3> <a name='Analysis_cluster'></a>


OUR Migrant MIGHT BE LIVING IN ANY OF THESE NEIGHBORHOODS IN DELHI, WE WILL MAKE CLUSTERS OF SIMILAR NEIGHBORHOODS AND COMPARE THESE CLUSTERS WITH THE NEIGHBORHOODS CLUSTER IN CITY OF TORONTO

* Our goal here is to find the neighborhoods with similar venues an combine them into clusters

* The most intuitive idea would be to find neighborhoods that have similar patterns of most frequently visited venues.

* This can be achieved by clustering the neighborhoods of the basis of the venues data we have acquired. Clustering is a predominant algorithm of unsupervised Machine Learning. It is used to segregate data entries in cluster depending of the similarity of their attributes, calculated by using the simple formula of euclidian distance.

* First we will orgainze the data of frequently visited venues in both the cities by adding the credentials like Borough name, latitude and longitude to our data set.

* This will be of help in visualization after cluster formation, so lets do that to both Delhi and Toronto !!
  
<img src="https://dimensionless.in/wp-content/uploads/2019/05/cover_cluster.jpg" width="400" height="300" />
   
   ----

##### First Merging the data for Toronto by adding Neighborhood Column and Borough Column corresponding to Normalized data

In [26]:
# merge toronto_grouped with toronto_data to add latitude/longitude and Borough name for each neighborhood

Toronto_Merged = Toronto_Data_Needed
Toronto_Merged = Toronto_Merged.join(Neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_Merged.drop('Postal code', axis=1, inplace=True)
Toronto_Merged.head()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,North York,Hillcrest Village,43.803762,-79.363452,Fast Food Restaurant,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Event Space,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
1,North York,"Fairview , Henry Farm , Oriole",43.778517,-79.346556,Clothing Store,Coffee Shop,Fast Food Restaurant,Kids Store,Bakery,Restaurant,Japanese Restaurant,Cosmetics Shop,Tea Room,Food Court,Bank,Chinese Restaurant,Chocolate Shop,Mobile Phone Shop,Metro Station
2,North York,Bayview Village,43.786947,-79.385975,Bank,Café,Chinese Restaurant,Japanese Restaurant,Women's Store,Event Space,Electronics Store,Dog Run,Discount Store,Diner,Food & Drink Shop,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
3,North York,"York Mills , Silver Hills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,North York,"Willowdale , Newtonbrook",43.789053,-79.408493,Piano Bar,Women's Store,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space


##### Similarly for Delhi, adding the respective columns back 

In [27]:
# merge delhiData with neighborhood venue data to add latitude/longitude and borough name for each neighborhood

delhi_merged = delhiData
delhi_merged = delhi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
delhi_merged.dropna(inplace=True)

delhi_merged.rename(columns={"latitude": "Latitude", "longitude": "Longitude"}, inplace=True)

delhi_merged.head()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,North West Delhi,Adarsh Nagar,28.714401,77.167288,Train Station,Gym / Fitness Center,Light Rail Station,Astrologer,Women's Store,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Fish & Chips Shop,Garden
1,North West Delhi,Ashok Vihar,28.699453,77.184826,Diner,Athletics & Sports,Bakery,Smoke Shop,Health & Beauty Service,Hotel Bar,Electronics Store,Donut Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
2,North West Delhi,Azadpur,28.707657,77.175547,Indian Restaurant,Bus Station,Pool Hall,Restaurant,Fish & Chips Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Frozen Yogurt Shop,Farmers Market,Falafel Restaurant
3,North West Delhi,Bawana,28.799660,77.032885,Playground,Women's Store,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Furniture / Home Store,Falafel Restaurant,Fabric Shop
5,North West Delhi,Dhaka,28.708698,77.205749,ATM,Pizza Place,Train Station,Furniture / Home Store,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Farmers Market,Fast Food Restaurant,Falafel Restaurant


#### Combining both Cities Venue data for further visualization and Analysis comparison on the Basis of K-Means Clusters
----

In [28]:
## Concatinating both the datas for furhter comparison

Frequent_Venues= pd.concat([Toronto_Merged, delhi_merged], axis=0, ignore_index=True)
print(Frequent_Venues.shape)
Frequent_Venues.sort_values(by='Neighborhood', axis = 0, ascending = True, inplace = True)
Frequent_Venues.reset_index(inplace=True)
Frequent_Venues.drop('index', axis=1,inplace=True)
Frequent_Venues=Frequent_Venues.drop_duplicates('Neighborhood',keep='last')
print(Frequent_Venues.shape)
Frequent_Venues.head()

(206, 19)
(191, 19)


,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,North West Delhi,Adarsh Nagar,28.714401,77.167288,Train Station,Gym / Fitness Center,Light Rail Station,Astrologer,Women's Store,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Fish & Chips Shop,Garden
1,Scarborough,Agincourt,43.794200,-79.262029,Skating Rink,Latin American Restaurant,Lounge,Breakfast Spot,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space
2,South Delhi,Alaknanda,28.529336,77.251632,Indian Restaurant,BBQ Joint,Steakhouse,New American Restaurant,Pizza Place,Coffee Shop,Food & Drink Shop,Restaurant,Middle Eastern Restaurant,French Restaurant,Food Truck,Food Court,Food,Flea Market,Fast Food Restaurant
3,East Delhi,Anand Vihar,28.641115,77.312502,Fast Food Restaurant,Pizza Place,Movie Theater,Multiplex,Women's Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Department Store,Falafel Restaurant
4,West Delhi,Ashok Nagar,28.636021,77.101822,Fast Food Restaurant,Ice Cream Shop,Café,Clothing Store,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Women's Store,Furniture / Home Store,Farmers Market


----

#### Merging one hot Coded / Normalized Data

##### We will now Merge both the Delhi Neighborhood Data and Toronto Neighborhood data so that we can make clusters of similar Neighborhoods by most common frequent Venues in both of them.

---
---

In [29]:
## Taking the one hot Encoded(normalized) data from both the cities and merging them together

Cluster_Data= pd.concat([Toronto_Grouped, delhi_grouped], join='inner',  axis=0, ignore_index=True)
Cluster_Data.sort_values(by='Neighborhood', axis = 0, ascending = True, inplace = True)
Cluster_Data.reset_index(inplace=True)
Cluster_Data.drop('index', axis=1,inplace=True)
print(Cluster_Data.shape)

Cluster_Data.head()


(189, 80)


,Neighborhood,Accessories Store,Airport,American Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Boutique,Breakfast Spot,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Golf Course,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Spa,Sporting Goods Shop,Steakhouse,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Video Game Store,Women's Store
0,Adarsh Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.090909,0.0,0.0,0.00,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,Anand Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.25,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,Ashok Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.2,0.0,0.2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.40,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0


#### Cleaning the Cities Venue Data on the basis of Merged Normalized Data
----

In [30]:
## When we added the Toronto's Venues for all neighbourhoods with Delhi's Venues for all neighborhoods, because of inner join 
##(continued) we might have lost some neighborhoods which didint have any Venues in common, such a neighborhood is not fit for 
##(continued) cluster comparsion Thus we will find the remaining Neighborhoods form Cluster_data and Keep only those Venues in Frequent Venues

Frequent_venues=pd.DataFrame()

for data in Cluster_Data['Neighborhood']:
  
    Frequent_common_Venues = Frequent_Venues[Frequent_Venues['Neighborhood'] == data]
    
    Frequent_venues=pd.concat([Frequent_venues, Frequent_common_Venues], axis=0, ignore_index=True)
    
print(Frequent_venues.shape)

Frequent_venues.head()

(189, 19)


,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,North West Delhi,Adarsh Nagar,28.714401,77.167288,Train Station,Gym / Fitness Center,Light Rail Station,Astrologer,Women's Store,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Fish & Chips Shop,Garden
1,Scarborough,Agincourt,43.794200,-79.262029,Skating Rink,Latin American Restaurant,Lounge,Breakfast Spot,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space
2,South Delhi,Alaknanda,28.529336,77.251632,Indian Restaurant,BBQ Joint,Steakhouse,New American Restaurant,Pizza Place,Coffee Shop,Food & Drink Shop,Restaurant,Middle Eastern Restaurant,French Restaurant,Food Truck,Food Court,Food,Flea Market,Fast Food Restaurant
3,East Delhi,Anand Vihar,28.641115,77.312502,Fast Food Restaurant,Pizza Place,Movie Theater,Multiplex,Women's Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Department Store,Falafel Restaurant
4,West Delhi,Ashok Nagar,28.636021,77.101822,Fast Food Restaurant,Ice Cream Shop,Café,Clothing Store,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Women's Store,Furniture / Home Store,Farmers Market


<h3> Applying the clustering algorithm </h3> <a name='K-means'> </a>

K-means algorithm is an iterative algorithm that tries to partition the dataset into K pre-defined distinct non-overlapping subgroups (clusters) where each data point belongs to only one group. It tries to make the inter-cluster data points as similar as possible while also keeping the clusters as different (far) as possible. It assigns data points to a cluster such that the sum of the squared distance between the data points and the cluster's centroid (arithmetic mean of all the data points that belong to that cluster) is at the minimum. The less variation we have within clusters, the more homogeneous (similar) the data points are within the same cluster.
 
---

In [31]:
# set number of clusters
kclusters = 7

Cluster_Data = Cluster_Data.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Cluster_Data)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15]

array([5, 5, 0, 4, 3, 5, 0, 5, 0, 0, 5, 0, 5, 5, 1], dtype=int32)

In [32]:
# add clustering labels
Frequent_venues.insert(0, 'Cluster Labels', kmeans.labels_)
Frequent_venues.head(10)

,Cluster Labels,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,5,North West Delhi,Adarsh Nagar,28.714401,77.167288,Train Station,Gym / Fitness Center,Light Rail Station,Astrologer,Women's Store,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Fish & Chips Shop,Garden
1,5,Scarborough,Agincourt,43.794200,-79.262029,Skating Rink,Latin American Restaurant,Lounge,Breakfast Spot,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space
2,0,South Delhi,Alaknanda,28.529336,77.251632,Indian Restaurant,BBQ Joint,Steakhouse,New American Restaurant,Pizza Place,Coffee Shop,Food & Drink Shop,Restaurant,Middle Eastern Restaurant,French Restaurant,Food Truck,Food Court,Food,Flea Market,Fast Food Restaurant
3,4,East Delhi,Anand Vihar,28.641115,77.312502,Fast Food Restaurant,Pizza Place,Movie Theater,Multiplex,Women's Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Department Store,Falafel Restaurant
4,3,West Delhi,Ashok Nagar,28.636021,77.101822,Fast Food Restaurant,Ice Cream Shop,Café,Clothing Store,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Women's Store,Furniture / Home Store,Farmers Market
5,5,North West Delhi,Ashok Vihar,28.699453,77.184826,Diner,Athletics & Sports,Bakery,Smoke Shop,Health & Beauty Service,Hotel Bar,Electronics Store,Donut Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
6,0,North West Delhi,Azadpur,28.707657,77.175547,Indian Restaurant,Bus Station,Pool Hall,Restaurant,Fish & Chips Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Frozen Yogurt Shop,Farmers Market,Falafel Restaurant
7,5,North East Delhi,Babarpur,28.687431,77.279755,Park,Gift Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
8,0,South Delhi,Badarpur,28.493170,77.303024,Indian Restaurant,IT Services,Train Station,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Furniture / Home Store,Falafel Restaurant
9,0,West Delhi,Bali Nagar,28.654138,77.128178,Indian Restaurant,Shopping Mall,Garden,Chinese Restaurant,Shoe Store,Ice Cream Shop,Antique Shop,Diner,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop



----

<h3> Visualizing Clusters </h3> <a name='VisualizingClusters'></a>

----
----

In [36]:
### This below code was used to generate the Lattitude and Longitude Values of Toronto City

address = 'North York, Toronto'

geolocator_T = Nominatim(user_agent="Toronto_explorer")
location_T = geolocator_T.geocode(address)
Latitude_T = location_T.latitude
Longitude_T = location_T.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(Latitude_T, Longitude_T))

# create map
map_clusters_1 = folium.Map(location=[latitude, longitude], zoom_start=11)
map_clusters_2 = folium.Map(location=[Latitude_T, Longitude_T], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Frequent_venues['Latitude'], Frequent_venues['Longitude'], Frequent_venues['Neighborhood'], Frequent_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters_1)
    
markers_colors = []
for lat, lon, poi, cluster in zip(Frequent_venues['Latitude'], Frequent_venues['Longitude'], Frequent_venues['Neighborhood'], Frequent_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters_2)
    


The geograpical coordinate of Toronto are 43.7543263, -79.44911696639593.


-----
#### Now, we can examine similarly Color coded clusters of city in there respective Maps

In [37]:
##Cluster Maps for Delhi
display(Markdown('### A folium map is being diplayed here which is mostly not visible on GITHUB '))
map_clusters_1

### A folium map is being diplayed here which is mostly not visible on GITHUB 

In [40]:
##Cluster Maps for Toronto
display(Markdown('### A folium map is being diplayed here which is mostly not visible on GITHUB '))
map_clusters_2

### A folium map is being diplayed here which is mostly not visible on GITHUB 


-----
<h3>Analyzing Clusters</h3><a name='Analyzing_Clusters'></a>

----

In [41]:
Cluster_1=Frequent_venues.loc[Frequent_venues['Cluster Labels'] == 0, Frequent_venues.columns[[1]+[2] + list(range(5, Frequent_venues.shape[1]))]]
Cluster_1.reset_index(inplace=True, drop=True)
Cluster_1.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,South Delhi,Alaknanda,Indian Restaurant,BBQ Joint,Steakhouse,New American Restaurant,Pizza Place,Coffee Shop,Food & Drink Shop,Restaurant,Middle Eastern Restaurant,French Restaurant,Food Truck,Food Court,Food,Flea Market,Fast Food Restaurant
1,North West Delhi,Azadpur,Indian Restaurant,Bus Station,Pool Hall,Restaurant,Fish & Chips Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Frozen Yogurt Shop,Farmers Market,Falafel Restaurant
2,South Delhi,Badarpur,Indian Restaurant,IT Services,Train Station,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Furniture / Home Store,Falafel Restaurant
3,West Delhi,Bali Nagar,Indian Restaurant,Shopping Mall,Garden,Chinese Restaurant,Shoe Store,Ice Cream Shop,Antique Shop,Diner,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
4,New Delhi,Barakhamba Road,Indian Restaurant,Asian Restaurant,Salon / Barbershop,Bar,BBQ Joint,Tea Room,Historic Site,Gastropub,Hotel,Hotel Bar,History Museum,Fast Food Restaurant,IT Services,Food Truck,Food Court


In [42]:
Cluster_2=Frequent_venues.loc[Frequent_venues['Cluster Labels'] == 1, Frequent_venues.columns[[1]+[2] + list(range(5, Frequent_venues.shape[1]))]]
Cluster_2.reset_index(inplace=True, drop=True)
Cluster_2.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,North York,Bayview Village,Bank,Café,Chinese Restaurant,Japanese Restaurant,Women's Store,Event Space,Electronics Store,Dog Run,Discount Store,Diner,Food & Drink Shop,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
1,Scarborough,"Birch Cliff , Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space
2,North Delhi,Civil Lines,Indian Restaurant,Coffee Shop,Café,Smoke Shop,Asian Restaurant,Chinese Restaurant,Hotel,Fabric Shop,Food,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Eastern European Restaurant
3,South Delhi,Dayanand Colony,Coffee Shop,Fast Food Restaurant,Indie Movie Theater,Café,Market,Restaurant,Flea Market,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Women's Store,Fish & Chips Shop,Frozen Yogurt Shop
4,South West Delhi,Delhi Cantonment,Indian Restaurant,Shopping Mall,Coffee Shop,Café,Dessert Shop,Flea Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [43]:
Cluster_3=Frequent_venues.loc[Frequent_venues['Cluster Labels'] == 2, Frequent_venues.columns[[1]+[2] + list(range(5, Frequent_venues.shape[1]))]]
Cluster_3.reset_index(inplace=True, drop=True)
Cluster_3.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,East Delhi,East Vinod Nagar,Hotel,Women's Store,Fish & Chips Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Garden,Farmers Market,Falafel Restaurant
1,South Delhi,Friends Colony,Hotel,Tea Room,Music Venue,Women's Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Fast Food Restaurant,Furniture / Home Store
2,Central Delhi,Jhandewalan,Hotel,Light Rail Station,Motorcycle Shop,High School,Women's Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store
3,East Delhi,Jhilmil Colony,Indian Restaurant,Hotel,Asian Restaurant,Juice Bar,Fish & Chips Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Farmers Market,Fast Food Restaurant
4,South Delhi,Jor Bagh,Hotel,Department Store,Metro Station,Bookstore,Historic Site,Chinese Restaurant,Event Space,Food,Fried Chicken Joint,French Restaurant,Food Truck,Donut Shop,Food Court,Food & Drink Shop,Flea Market


In [44]:
Cluster_4=Frequent_venues.loc[Frequent_venues['Cluster Labels'] == 3, Frequent_venues.columns[[1]+[2] + list(range(5, Frequent_venues.shape[1]))]]
Cluster_4.reset_index(inplace=True, drop=True)
Cluster_4.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,West Delhi,Ashok Nagar,Fast Food Restaurant,Ice Cream Shop,Café,Clothing Store,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Women's Store,Furniture / Home Store,Farmers Market
1,Scarborough,"Malvern , Rouge",Fast Food Restaurant,Women's Store,History Museum,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space
2,North Delhi,Pul Bangash,Fast Food Restaurant,Indian Restaurant,Metro Station,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Fish & Chips Shop,Garden,Farmers Market,Falafel Restaurant
3,West Delhi,Punjabi Bagh,Fast Food Restaurant,Pizza Place,Garden Center,Sandwich Place,Gym / Fitness Center,Fish & Chips Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Women's Store,Frozen Yogurt Shop,Farmers Market
4,West Delhi,Tilak Nagar,Fast Food Restaurant,ATM,Restaurant,Light Rail Station,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Furniture / Home Store,Farmers Market,Falafel Restaurant


In [45]:
Cluster_5=Frequent_venues.loc[Frequent_venues['Cluster Labels'] == 4, Frequent_venues.columns[[1]+[2] + list(range(5, Frequent_venues.shape[1]))]]
Cluster_5.reset_index(inplace=True, drop=True)
Cluster_5.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,East Delhi,Anand Vihar,Fast Food Restaurant,Pizza Place,Movie Theater,Multiplex,Women's Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Department Store,Falafel Restaurant
1,South Delhi,Chittaranjan Park,Market,Fast Food Restaurant,Pizza Place,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Women's Store,Furniture / Home Store,Farmers Market,Falafel Restaurant
2,Scarborough,"Clarks Corners , Tam O'Shanter , Sullivan",Pharmacy,Pizza Place,Coffee Shop,Fried Chicken Joint,Chinese Restaurant,Noodle House,Italian Restaurant,Intersection,Bank,Fast Food Restaurant,Gas Station,Thai Restaurant,Frozen Yogurt Shop,Dim Sum Restaurant,Construction & Landscaping
3,North West Delhi,Dhaka,ATM,Pizza Place,Train Station,Furniture / Home Store,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Farmers Market,Fast Food Restaurant,Falafel Restaurant
4,North East Delhi,Dilshad Garden,Indian Restaurant,Diner,Pizza Place,Light Rail Station,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Furniture / Home Store,Farmers Market


In [46]:
Cluster_6=Frequent_venues.loc[Frequent_venues['Cluster Labels'] == 5, Frequent_venues.columns[[1]+ [2]+ list(range(5, Frequent_venues.shape[1]))]]
Cluster_6.reset_index(inplace=True, drop=True)
Cluster_6.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,North West Delhi,Adarsh Nagar,Train Station,Gym / Fitness Center,Light Rail Station,Astrologer,Women's Store,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Fish & Chips Shop,Garden
1,Scarborough,Agincourt,Skating Rink,Latin American Restaurant,Lounge,Breakfast Spot,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space
2,North West Delhi,Ashok Vihar,Diner,Athletics & Sports,Bakery,Smoke Shop,Health & Beauty Service,Hotel Bar,Electronics Store,Donut Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
3,North East Delhi,Babarpur,Park,Gift Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
4,Central Delhi,Bara Hindu Rao,Afghan Restaurant,ATM,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Fish & Chips Shop,Garden Center,Fast Food Restaurant,Farmers Market


In [47]:
Cluster_7=Frequent_venues.loc[Frequent_venues['Cluster Labels'] == 6, Frequent_venues.columns[[1] + [2] + list(range(5, Frequent_venues.shape[1]))]]
Cluster_7.reset_index(inplace=True, drop=True)
Cluster_7.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,North East Delhi,Kabir Nagar,Park,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Furniture / Home Store,Falafel Restaurant,Fabric Shop
1,North East Delhi,Maujpur,Park,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Farmers Market,Furniture / Home Store,Falafel Restaurant,Fabric Shop
2,North West Delhi,Narela,Electronics Store,Women's Store,Fish & Chips Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Garden,Farmers Market,Falafel Restaurant


----
Analysing these clusters we can find relocation neighborhoods in Toronto for migrater's native locations/Neighborhoods in Delhi.

----

<h3> RECOMENDATIONS</h3> <a name='recommendation'></a>


#### In this section:
    
    We will try to Recommend the neighborhoods in North York,Toronto or Scarborough, Toronto for neighborhoods native to our migrants in Delhi 



In [48]:
## Defining a function to split our clusters into differnt cluster for Delhi and Toronto

def return_Seprated_clusters(Cluster):

    Cluster_Toronto=pd.DataFrame()
    Cluster_Delhi=pd.DataFrame()
    
    for data in Toronto_Merged['Neighborhood']:
  
        Common_Toronto= Cluster[Cluster['Neighborhood'] == data]
    
        Cluster_Toronto=pd.concat([Cluster_Toronto, Common_Toronto], axis=0, ignore_index=True)
        Cluster_Toronto=Cluster_Toronto[['Borough', 'Neighborhood']]
            
    
    for data in delhi_merged['Neighborhood']:
  
        Common_Delhi= Cluster[Cluster['Neighborhood'] == data]
    
        Cluster_Delhi=pd.concat([Cluster_Delhi, Common_Delhi], axis=0, ignore_index=True)
        Cluster_Delhi=Cluster_Delhi[['Borough', 'Neighborhood']]
        
    return(Cluster_Toronto, Cluster_Delhi)

In [49]:
#Obtaing all the Serprated Clusters in different dataframes

Cluster_1_Toronto, Cluster_1_Delhi= return_Seprated_clusters(Cluster_1)
Cluster_2_Toronto, Cluster_2_Delhi= return_Seprated_clusters(Cluster_2)
Cluster_3_Toronto, Cluster_3_Delhi= return_Seprated_clusters(Cluster_3)
Cluster_4_Toronto, Cluster_4_Delhi= return_Seprated_clusters(Cluster_4)
Cluster_5_Toronto, Cluster_5_Delhi= return_Seprated_clusters(Cluster_5)
Cluster_6_Toronto, Cluster_6_Delhi= return_Seprated_clusters(Cluster_6)
Cluster_7_Toronto, Cluster_7_Delhi= return_Seprated_clusters(Cluster_7)


In [50]:
# a fuction defintition to display dataframes next to each other

from IPython.display import display_html, Markdown

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)




---
#### Recommendation for native neighborhoods in Delhi and suggested Neighborhoods in Toronto
---

In [51]:
## For Cluster_1
display(Markdown('### Recommendation 1'))
print(" ")
display_side_by_side(Cluster_1_Delhi, Cluster_1_Toronto)

### Recommendation 1

,Borough,Neighborhood
0,North West Delhi,Azadpur
1,North West Delhi,Kohat Enclave
2,North West Delhi,Pitam Pura
3,North West Delhi,Rani Bagh
4,North Delhi,Chandni Chowk
5,North Delhi,Dariba Kalan
6,North Delhi,Chandni Chowk
7,Central Delhi,Daryaganj
8,New Delhi,Barakhamba Road
9,New Delhi,Connaught Place


----

----
----

In [52]:
## For Cluster_2

display(Markdown('### Recommendation 2'))
print(" ")
display_side_by_side(Cluster_2_Delhi, Cluster_2_Toronto)

### Recommendation 2

,Borough,Neighborhood
0,North West Delhi,Kingsway Camp
1,North Delhi,Shakti Nagar
2,North Delhi,Civil Lines
3,North Delhi,Shakti Nagar
4,New Delhi,Khan Market
5,South Delhi,Dayanand Colony
6,South Delhi,Malviya Nagar
7,South Delhi,Neeti Bagh
8,South Delhi,New Friends Colony
9,South Delhi,Sarvodaya Enclave


----
----
---

In [53]:
## For Cluster_3

display(Markdown('### Recommendation 3'))
print(" ")

display_side_by_side(Cluster_3_Delhi, Cluster_3_Toronto)

### Recommendation 3

,Borough,Neighborhood
0,North West Delhi,Rithala
1,Central Delhi,Paharganj
2,Central Delhi,Jhandewalan
3,Central Delhi,Karol Bagh
4,Central Delhi,Paharganj
5,New Delhi,Pragati Maidan
6,East Delhi,East Vinod Nagar
7,East Delhi,Jhilmil Colony
8,South Delhi,Friends Colony
9,South Delhi,Jor Bagh


----
-----
-----

In [54]:
## For Cluster_4

display(Markdown('### Recommendation 4'))
print(" ")
display_side_by_side(Cluster_4_Delhi, Cluster_4_Toronto)

### Recommendation 4

,Borough,Neighborhood
0,North Delhi,Pul Bangash
1,West Delhi,Ashok Nagar
2,West Delhi,Punjabi Bagh
3,West Delhi,Tilak Nagar
,Borough,Neighborhood
0,Scarborough,"Malvern , Rouge"


----
----
----

In [55]:
##For Cluster_5

display(Markdown('### Recommendation 5'))
print(" ")
display_side_by_side(Cluster_5_Delhi, Cluster_5_Toronto)


### Recommendation 5

,Borough,Neighborhood
0,North West Delhi,Dhaka
1,North West Delhi,Model Town
2,North Delhi,Kamla Nagar
3,North Delhi,Kashmiri Gate
4,North Delhi,Tis Hazari
5,North East Delhi,Dilshad Garden
6,East Delhi,Geeta Colony
7,East Delhi,Krishna Nagar
8,East Delhi,Preet Vihar
9,East Delhi,Anand Vihar


-----
----
-----

In [56]:
##For Cluster_6

display(Markdown('### Recommendation 6'))
print(" ")
display_side_by_side(Cluster_6_Delhi, Cluster_6_Toronto)

### Recommendation 6

,Borough,Neighborhood
0,North West Delhi,Adarsh Nagar
1,North West Delhi,Ashok Vihar
2,North West Delhi,Bawana
3,North West Delhi,Jahangirpuri
4,North West Delhi,Keshav Puram
5,Central Delhi,Bara Hindu Rao
6,North Delhi,Chawri Bazaar
7,North Delhi,Gulabi Bagh
8,North Delhi,Lahori Gate
9,North Delhi,Nehru Vihar


----
----
----

In [57]:
##For Cluster_7

display(Markdown('### Recommendation 7'))
print(" ")

display_side_by_side(Cluster_7_Delhi, Cluster_7_Toronto)


### Recommendation 7

,Borough,Neighborhood
0,North West Delhi,Narela
1,North East Delhi,Kabir Nagar
2,North East Delhi,Maujpur
,Borough,Neighborhood


-----
-----
----

<h3> Results and Discussions</h3><a name='Results'></a>

Finally all neighborhoods which fall in one cluster arranged on the basis of similarity in nearby venues were found and displayed side by side

Our Analysis was done on over 188 neighborhoods Combined in Delhi and Toronto, containing 100 venues each within a radius of 500 meters. We segregated these neighborhoods on the basis of Frequency of venues nearby. 7 clusters were obtained, each having a unique collection of Venues. Since, we were focused on finding heuristic relocation location for one neighborhood to other we found many probable neighborhood in Toronto which appeals the same as native neighborhood in Delhi.

The neighborhoods recommendation obtained here are not completely accurate. This is due to the limitations in the dataset used in the project. Due to lack of cross referencing sources, we may have missed a few neighborhoods from our consideration. The foursquare API does not contain, or does not rely, a comprehensive dataset about the venues present in Delhi. Surely, in a city like Delhi with a population of over 19 million, there are much more criteria which should have been taken into account. Also in this study no cultural, and social background of particular parties were taken into account thus making this study over-generalized. 

On the Conclusion: The resulted neighborhood recommendation in Toronto can act as a good starting point for figuring out which location might suit best. Each person then by his personal preferences further refine the study and take benefits of initial iterations provided in this Project